In [1]:
import pandas as pd
import time
import random
import math
import mmh3

def str_to_MinHash(str1, q, seed=0):
    return min([mmh3.hash(str1[i:i + q], seed) for i in range(len(str1) - q + 1)])

def frequent2(temp, L, t):
    return {k: v for (k, v) in temp.items() if v/L >= t}

def matching():
    global tp, fp, pairsNo, L1, q
    for index2 in range(nbS, nbS + offsetB): 
        if index2 > len(df2) - 1:
            return True

        rr = df2.iloc[index1]
        ncid = rr["id"]
        first_name = rr["first_name"]
        last_name = rr["last_name"]
        registr_dt = rr["registr_dt"]
        age_at_year_end = rr["age_at_year_end"]
        srec = first_name + " " + last_name + " " + str(registr_dt) + " " + str(age_at_year_end)

        temp = dict()
        indices = [random.randrange(0, L) for i in range(L1)]
        matchingPairs = {}
        for l in indices:
            key = str(str_to_MinHash(srec.lower(), q, l))
            d = dictB[l]
            if key in d:
                ids = d[key]
                for id in ids:
                    if id in temp:
                        temp[id] += 1
                        if temp[id] / L1 >= t:
                            matchingPairs[id] = 1
                    else:
                        temp[id] = 1
                        
        for id in matchingPairs.keys():
            idDBLP = id
            pairsNo += 1
            if idDBLP in truthD:
                ids = truthD[idDBLP]
                for id in ids:
                    if id == ncid:
                        tp += 1
                        break
            else:
                fp += 1

    return False




In [ ]:
def elimina_elementos_dentro_dictB(array_para_descarte, array_para_descarte_igual):
    del array_para_descarte
    del array_para_descarte_igual


In [ ]:
df1 = pd.read_csv("../00-datasets/ncvoter42.csv", sep=",", encoding="utf-8", keep_default_na=False)
df2 = pd.read_csv("../00-datasets/perturbed_recordsNC_voter.csv", sep=",", encoding="utf-8", keep_default_na=False)
truth = pd.read_csv("../00-datasets/ground_truthNC_voter.csv", sep=",", encoding="utf-8", keep_default_na=False)
truthD = dict()
for i, r in truth.iterrows():
    novoId = r["novoId"]
    antigoId = r["antigoId"]
    if antigoId in truthD:
        ids = truthD[antigoId]
        ids.append(novoId)
    else:
        truthD[antigoId] = [novoId]

t = 0.5
TP = 40893
eps = 0.1
w = 1000
delta = 0.1
L = math.ceil(math.log(1 / delta) / (2 * (eps ** 2)))
eps = 0.01
L1 = int(1 / (2 * eps))
print("L=", L, "L1=", L1)
q = 2
dictB = [dict() for l in range(L)]
dictB_igual = [dict() for l in range(L)]
tp = 0
fp = 0
pairsNo = 0
nbS = 1
naS = 1
offsetA = 50
offsetB = 50
blockingTime = 0
matchingTime = 0


tempoQueFoiInseridoNaEstrutura = 0 # esse é o valor que sera colocado junto com o id, seria a posiçaõ do elemento no array
elementos_para_descarte = 50 # esse é a quantidade de elementos que serão descartados
acompanhamentoIndicePorBloco = {} #essa estrutura é um indice que vai ser usada para saber qual a posição do elemento que será descartado
tamanhoDosBlocos = {}
min_descarte = int(elementos_para_descarte * 0.5)
max_descarte = int(elementos_para_descarte * 1.5)

while True:
    st = time.time()
    for index1 in range(naS, naS + offsetA):
        if index1 >= len(df1):
            break
        rr = df1.iloc[index1, 0:69]
        
        ncid = rr["ncid"]
        first_name = rr["first_name"]
        last_name = rr["last_name"]
        registr_dt = rr["registr_dt"]
        age_at_year_end = rr["age_at_year_end"]
        srec = first_name + " " + last_name + " " + str(registr_dt) + " " + str(age_at_year_end)
        key = ""
        
        for l in range(L):
            key = str(str_to_MinHash(srec.lower(), 2, l))
            d = dictB[l]
            d_igual = dictB_igual[l]
            if key in d:
                ids = d[key]
                ids_igual = d_igual[key]
                tamanho_atual = tamanhoDosBlocos[(key, l)]
                if len(ids) < tamanho_atual:
                    ids.append(ncid)
                    ids_igual.append(tempoQueFoiInseridoNaEstrutura)
                else:
                    elimina_elementos_dentro_dictB(ids, ids_igual)
                    print(len(ncid))
                    ids.append(ncid)
                    ids_igual.append(tempoQueFoiInseridoNaEstrutura)
    
            else:
                d[key] = [ncid]
                d_igual[key] = [tempoQueFoiInseridoNaEstrutura]
                acompanhamentoIndicePorBloco[(key, l)] = [0, elementos_para_descarte]
                tamanhoDosBlocos[(key, l)] = w
            
        tempoQueFoiInseridoNaEstrutura += 1
    
    end = time.time()

    blockingTime += (end - st)
    st = time.time()
    termination = matching()  
    end = time.time()
    matchingTime += (end - st)
    if termination:
            break

    nbS += offsetB
    naS += offsetA

print("blocking time (in mins)", blockingTime / 60)
print("matching time (in mins)", matchingTime / 60)
if tp + fp > 0:
    print("TP=", tp, "Recall=", tp / TP, "Precision=", tp / (tp + fp), "pairsNo=", pairsNo)

L= 116 L1= 50
<class 'list'>
[0, 3, 4, 5, 6, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 67, 68, 69, 71, 72, 73, 74, 75, 78, 79, 80, 81, 82, 83, 84, 85, 86, 89, 90, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 106, 107, 108, 109, 110, 111, 113, 114, 115, 116, 117, 118, 119, 120, 123, 124, 125, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 141, 144, 146, 147, 148, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 181, 182, 183, 184, 186, 187, 188, 189, 190, 191, 193, 194, 195, 196, 197, 200, 201, 203, 204, 205, 206, 208, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 221, 222, 225, 226, 227, 228, 229, 231, 233, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 251, 252, 253, 254, 256, 257, 258, 259, 26

KeyboardInterrupt: 

```
L= 116 L1= 50
blocking time (in mins) 0.7452942967414856
matching time (in mins) 7.856352432568868
TP= 34941 Recall= 0.8544494167705964 Precision= 0.23017938194586263 pairsNo= 267653
```